In [ ]:
!pip install langchain_openai
!pip install langchain_core
!pip install langchain_huggingface

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
# from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
import torch
import pandas as pd
import transformers
import os

import datetime
import os
import json

/opt/conda/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3553: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [2]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3.5-mini-instruct", trust_remote_code=True)
# model = AutoModelForCausalLM.from_pretrained("microsoft/Phi-3.5-mini-instruct", trust_remote_code=True)

In [3]:
df_base = pd.read_csv("results_of_base_model.csv")

In [ ]:
df_base

In [6]:
# # Model to fine-tune
# model = AutoModelForCausalLM.from_pretrained(
#     "microsoft/Phi-3.5-mini-instruct",
#     torch_dtype=torch.float16,
#     load_in_4bit=True
# )

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`low_cpu_mem_usage` was None, now default to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
from peft import LoraConfig, PeftModel, get_peft_model

In [6]:
base_model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3.5-mini-instruct",
    return_dict=True,
    torch_dtype=torch.bfloat16,
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
PATH_MODEL = "model/checkpoint-1248"

model = PeftModel.from_pretrained(base_model, PATH_MODEL)
model = model.merge_and_unload()

In [8]:
def get_all_txt_contents_as_list(directory_path):
    all_contents = []
    
    # Loop through all files in the given directory
    for filename in os.listdir(directory_path):
        # Check if the file is a .txt file
        if filename.endswith(".txt"):
            file_path = os.path.join(directory_path, filename)
            
            try:
                # Open and read the .txt file
                with open(file_path, 'r', encoding='utf-8') as file:
                    content = file.read()
                    all_contents.append(content)  # Add content to the list
            except Exception as e:
                print(f"Error reading {file_path}: {str(e)}")
    
    return all_contents

In [9]:
def get_all_txt_contents_from_folders(parent_directory):
    all_txt_contents = []
    
    # Loop through each folder in the parent directory
    for folder_name in os.listdir(parent_directory):
        folder_path = os.path.join(parent_directory, folder_name)
        
        # Check if it is a directory
        if os.path.isdir(folder_path):
            # Call the function to read all .txt files in this folder
            folder_contents = get_all_txt_contents_as_list(folder_path)
            
            # Append each content with the folder name
            for content in folder_contents:
                all_txt_contents.append({
                    "folder": folder_name,
                    "content": content
                })
    
    return all_txt_contents

In [10]:
parent_directory = "lisa_sheets_translated"
all_txt_contents = get_all_txt_contents_from_folders(parent_directory)

In [15]:
all_txt_contents[0]["folder"] == 'IC-345'

False

In [11]:
import json

with open('test_folders.json', 'r') as file:
    test_folders = json.load(file)

In [42]:
df_base['id'].str.extract(r'OIC-(\d+)', expand=False).value_counts()

id
049    26
076    20
050    17
005    15
019    13
328    12
347    11
283     9
257     8
Name: count, dtype: int64

In [12]:
test_txt_contents = [x for x in all_txt_contents if x['folder'] in test_folders]

In [13]:
test_folders = ['IC-049', 'IC-076', 'IC-050', 'IC-005', 'IC-019', 'IC-328', 'IC-347', 'IC-283', 'IC-257']

test_txt_contents = [x for x in all_txt_contents if x['folder'] in test_folders]

In [14]:
model.to("cuda")

Phi3ForCausalLM(
  (model): Phi3Model(
    (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x Phi3DecoderLayer(
        (self_attn): Phi3SdpaAttention(
          (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (qkv_proj): Linear(in_features=3072, out_features=9216, bias=False)
          (rotary_emb): Phi3LongRoPEScaledRotaryEmbedding()
        )
        (mlp): Phi3MLP(
          (gate_up_proj): Linear(in_features=3072, out_features=16384, bias=False)
          (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
          (activation_fn): SiLU()
        )
        (input_layernorm): Phi3RMSNorm((3072,), eps=1e-05)
        (resid_attn_dropout): Dropout(p=0.0, inplace=False)
        (resid_mlp_dropout): Dropout(p=0.0, inplace=False)
        (post_attention_layernorm): Phi3RMSNorm((3072,), eps=1e-05)
      )
    )
    (norm): Phi3RMSNorm((3072,)

In [15]:
import re
def remove_double_bracketed_text(text):
    # Use regex to find and remove text within {{...}}
    cleaned_text = re.sub(r"\{\{.*?\}\}", "", text, flags=re.DOTALL)
    return cleaned_text.strip()

In [16]:
system_instruction = """
Based on the following educational content, generate a multiple-choice question with four answer 
options where only one is correct. The question should assess understanding of the main ideas, 
and the options should be clear, informative, and relevant. Ensure that the distractors (incorrect options) 
follow a logical but incorrect interpretation, based on common misconceptions or misunderstandings of the topic.
Answer options must be as short as possible.
"""

def get_response_from_phi3(lisa_sheet):
    inputs = None
    outputs = None
    try:
        full_prompt = f"<|system|>{system_instruction}<|end|>\n\n<|user|>{lisa_sheet}<|end|>\n\n<|assistant|>"
        
        inputs = tokenizer(full_prompt, return_tensors="pt").to("cuda")
        with torch.no_grad():
            outputs = model.generate(inputs.input_ids, max_length=4096, temperature=0.7, do_sample=True)
        
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    except Exception as error:
        print("An exception occurred:", error)
    finally:
        del inputs  # Free up GPU memory used by inputs
        del outputs  # Free up GPU memory used by outputs
        torch.cuda.empty_cache()
    
    return generated_text

In [35]:
mcqs = []


for index, item in enumerate(test_txt_contents[:20]):
    if index % 10 == 0:
        print(f'{index}, {datetime.datetime.now()}')
    # Extract content and folder from the item
    content = item["content"]
    folder = item["folder"]

    # Process content to create MCQ structure
    lisa_sheet_content = remove_double_bracketed_text(content)
    phi3_mcq = get_response_from_phi3(lisa_sheet_content)

    # Add content and folder to the MCQ
    # phi3_mcq['content'] = content
    # phi3_mcq['folder'] = folder

    # Append the updated MCQ to the mcqs list
    mcqs.append(phi3_mcq)

0, 2024-12-16 15:16:34.121771
10, 2024-12-16 15:19:56.369803


# Structure output

In [17]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain_huggingface.llms import HuggingFacePipeline

In [18]:
parser_model = ChatOpenAI(model="gpt-4o", temperature = 0.7, api_key = "sk-wuniLI4FhpnCoM-4H7usEfIktpDoy0YOiCE-EIHzmOT3BlbkFJWg0Ky_60c07TCiGODeOa8_6-HqYOPu4YwbHSGAsH0A")

In [19]:
class MCQQuestion(BaseModel):
    question: str = Field(description="The multiple-choice question")
    option_a: str = Field(description="The first answer option labeled 'A'")
    option_b: str = Field(description="The second answer option labeled 'B'")
    option_c: str = Field(description="The third answer option labeled 'C'")
    option_d: str = Field(description="The fourth answer option labeled 'D'")
    correct_option: str = Field(description="This consists only a letter of correct option")

mcq_parser = JsonOutputParser(pydantic_object=MCQQuestion)

prompt_template = PromptTemplate(
    template="{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": mcq_parser.get_format_instructions()},
)

chain = prompt_template | parser_model | mcq_parser

def structurize_lisa_sheet_response(raw_response):
    return chain.invoke({"query": raw_response})

In [20]:
system_instruction = """
Based on the following educational content, generate a multiple-choice question with four answer 
options where only one is correct. The question should assess understanding of the main ideas, 
and the options should be clear, informative, and relevant. Ensure that the distractors (incorrect options) 
follow a logical but incorrect interpretation, based on common misconceptions or misunderstandings of the topic.
Answer options must be as short as possible.
"""

In [21]:
def get_response_from_phi3(lisa_sheet):
    inputs = None
    outputs = None
    try:
        full_prompt = f"<|system|>{system_instruction}<|end|>\n\n<|user|>{lisa_sheet}<|end|>\n\n<|assistant|>"
        
        inputs = tokenizer(full_prompt, return_tensors="pt").to("cuda")
        with torch.no_grad():
            outputs = model.generate(inputs.input_ids, max_length=4096, temperature=0.7, do_sample=True)
        
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    except Exception as error:
        print("An exception occurred:", error)
    finally:
        del inputs  # Free up GPU memory used by inputs
        del outputs  # Free up GPU memory used by outputs
        torch.cuda.empty_cache()
    
    return generated_text

In [22]:
mcqs = []

for index, item in enumerate(test_txt_contents):
    if index % 10 == 0:
        print(f'{index}, {datetime.datetime.now()}')
    try:
        # Extract content from the dictionary
        content = item["content"]
        
        # Process content to create MCQ structure
        lisa_sheet_content = remove_double_bracketed_text(content)
        phi3_mcq = get_response_from_phi3(lisa_sheet_content)
        phi3_mcq = structurize_lisa_sheet_response(phi3_mcq)

        # Structure the response as specified
        structured_mcq = {
            "question": phi3_mcq.get("question", ""),
            "option_a": phi3_mcq.get("option_a", ""),
            "option_b": phi3_mcq.get("option_b", ""),
            "option_c": phi3_mcq.get("option_c", ""),
            "option_d": phi3_mcq.get("option_d", ""),
            "correct_option": phi3_mcq.get("correct_option", ""),
            "index": index,
            "folder": item["folder"]
        }
        
        # Append structured MCQ to the list
        mcqs.append(structured_mcq)
        
    except Exception as error:
        print("An exception occurred:", error)

0, 2024-12-16 18:58:59.908327


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


10, 2024-12-16 19:01:34.681010
20, 2024-12-16 19:03:58.649287
30, 2024-12-16 19:06:39.192298
40, 2024-12-16 19:09:26.784134
50, 2024-12-16 19:12:08.411952
60, 2024-12-16 19:14:29.494723
70, 2024-12-16 19:16:55.643558
80, 2024-12-16 19:19:28.340670
90, 2024-12-16 19:22:08.480553
100, 2024-12-16 19:24:36.175707
110, 2024-12-16 19:26:52.829720
120, 2024-12-16 19:29:01.631361
130, 2024-12-16 19:31:10.570173


In [23]:
mcqs

[{'question': 'What are the possible consequences of disciplinary action against a doctor, excluding criminal prosecution?',
  'option_a': 'A warning, a suspended sentence, and a temporary ban from practicing medicine.',
  'option_b': 'A warning, a reprimand, a temporary suspension, and potential removal from the medical register.',
  'option_c': 'A fine, community service, and a permanent ban from practicing medicine.',
  'option_d': 'A reprimand, a permanent ban, and imprisonment.',
  'correct_option': 'B',
  'index': 0,
  'folder': 'IC-005'},
 {'question': 'What distinguishes a therapeutic hazard from an error in medical practice?',
  'option_a': 'Therapeutic hazards are caused by negligent behavior of healthcare professionals.',
  'option_b': 'Therapeutic hazards are unforeseen events that occur without any responsibility on the part of healthcare professionals, linked to the inherent risks of medical acts.',
  'option_c': 'Therapeutic hazards are always preventable through proper 

In [24]:
with open('disclosure_tuned_dpo_131.json', 'w') as f:
    json.dump(mcqs, f)